### Web-scrape for horse results

In [ ]:
import os
import sys
import requests
import urllib
import urllib.request
from bs4 import BeautifulSoup
from csv import writer
import re
import pandas as pd

### Setting path and primary URL

In [ ]:
PATH = "usr/local/bin/chromedriver"
URL = "https://tnetwork.trakus.com/tnet/t_NYRA.aspx?EventID=205193&Date=7/10/2022&Type=TBRED&Venue=23&DisplayType=1"

### Importing URLs

In [ ]:
urls = pd.read_csv("urllist.csv")
urls_list = list(urls)
urls_series = pd.Series(urls_list)

### Scraping tables from URLS

In [ ]:
def scrape_table(URL):
    #Scrapes the table from the trakus website. 
    #Returns pandas df - will still require some cleaning. 
    dfs = pd.read_html(URL)
    df = pd.DataFrame(dfs[4]) # all the required data is in the fourth level of the output.
    df = df.iloc[:,[2,3,4,5,7,8]]
    df.columns = ["Horse name", "Start", "1/4", "1/2", "3/4", "Finish"]
    
    return df

In [ ]:
df = scrape_table(urls_list[0])
df.head()

### Cleaning tables

In [ ]:
def clean_table(df):
    df["Finish_position"] = df.index + 1
    df["Finishing time"] = df.Finish.str.split()
    times = []
    for i in df["Finishing time"]:
        times.append(i[-1])
    df["Finishing time"] = times
    
    return df

In [ ]:
output = clean_table(df)
output.head()

### Getting race numbers

Race numbers are helf in the gif names. Srape all gif names. 

- Example gif name: src="images/HorseRacingTiles/NYRA/4.gif" - where 4 is the horse number. 
- The images will be scraped in order, so we can append the dataframes with the ordered list. 

In [ ]:
def extract_race_number(url):
    img_tags = []
    race_numbers = []
    baseUrl = requests.get(url)
    soup = BeautifulSoup(baseUrl.text, 'html.parser')
    allImgs = soup.findAll('img')
    
    for i in allImgs:
        if "images/HorseRacingTiles/NYRA" in str(i):
            
            img_tags.append(str(i))
    
    for i in img_tags:
        m = re.search('src="images/HorseRacingTiles/NYRA/(.+?).gif', i)
        if m:
            found = m.group(1)
        race_numbers.append(str(found))
        
    return race_numbers
    
    

In [ ]:
extract_race_number(urls_list[0])

In [ ]:
### Extracting race date from url

In [ ]:
def get_date(url):
    m = re.search('Date=(.+?)&Type', url)
    if m:
        found = m.group(1)
    
    return found

get_date(urls_list[1])


In [ ]:
urls_list[1]

### Extracting Event ID

In [ ]:
def get_event_id(url):
    m = re.search('EventID=(.+?)&Date', url)
    if m:
        found = m.group(1)
    
    return found

get_event_id(urls_list[1])

### Full scrape

In [ ]:
def full_scrape(url):
    df = scrape_table(url)
    clean_df = clean_table(df)
    race_numbers = extract_race_number(url)
    clean_df["program_number"] = race_numbers
    clean_df["race_date"] = get_date(url)
    clean_df["event_id"] = get_event_id(url)

    
    return clean_df

df = full_scrape(urls_list[1])
df.head()

In [ ]:
df

### Collating all datasets

In [ ]:
urls_list = urls_list[:10]
urls_list

In [181]:
def collate_datasets(list_of_urls):
    df = full_scrape(list_of_urls[0])
    
    for i in list_of_urls[1:]:
        df2 = full_scrape(i)
        df = pd.concat([df, df2],ignore_index=True)
    
    return df

dfc = collate_datasets(urls_list[:10])
dfc.head()

,Horse name,Start,1/4,1/2,3/4,Finish,Finish_position,Finishing time,program_number,race_date,event_id
0,Jc's Shooting Star,5,42 23.77,43 47.92,25 3/4 1:12.89,1Neck 1:19.52,1,1:19.52,5,1/1/2019,149756
1,Sounds Delicious,11/4,13/4 23.23,11 1/2 47.35,11 1/4 1:12.69,26 3/4 1:19.56,2,1:19.56,1,1/1/2019,149756
2,Crimson Frost,41/4,5 24.06,5 48.42,41 1/4 1:14.05,31 1/2 1:20.74,3,1:20.74,2,1/1/2019,149756
3,Friend of Liberty,2Head,21/4 23.33,23/4 47.60,31/2 1:13.95,41 1/4 1:21.05,4,1:21.05,3,1/1/2019,149756
4,Bobby's Song,31/4,32 3/4 23.36,31 47.75,5 1:14.27,5 1:21.31,5,1:21.31,4,1/1/2019,149756


## Translating Event ID to Race Number

To convert event id to race numbers we take advantage of the fact that the data is grouped and ordered by date and race number in ascending order. Moreover, races are held ONLY at a single venue for each date. By considering each race day as a unique group, the unique event id's (starting from the smallest and assigned race number 1) are incremented by 1. The individual slices are then concatenated to create the final dataframe.

In [184]:
# convert event_id to race number 

racedate= dfc['race_date'].unique()
final = pd.DataFrame()

for date in racedate:
    
    bydates = dfc[dfc['race_date']== date].copy()
    ids = bydates['event_id'].unique()
    n = 1
    
    for i in ids:
       
        bydates.loc[bydates['event_id']==i,"race_number"] = str(n)
        n += 1 # increment race number
    
    final = pd.concat([new_df,bydates]) # build dataframe
        

final[20:40]

,Horse name,Start,1/4,1/2,3/4,Finish,Finish_position,Finishing time,program_number,race_date,event_id,race_number
20,Small Bear,51/4,5Head 24.62,6 49.09,3Head 1:40.86,1Neck 1:54.02,1,1:54.02,1,1/1/2019,149759,4.0
21,Holiday Bonus,6,31/2 24.24,3Head 48.79,11 3/4 1:40.51,2Neck 1:54.07,2,1:54.07,3,1/1/2019,149759,4.0
22,Backsideofthemoon,21/4,6 24.65,51 3/4 48.83,42 1/2 1:40.89,33 3/4 1:54.11,3,1:54.11,6,1/1/2019,149759,4.0
23,Hit It Once More,4Head,21 3/4 23.97,23 48.30,21/4 1:40.82,47 1:54.85,4,1:54.85,1A,1/1/2019,149759,4.0
24,Roaming Union,1Neck,11/2 23.89,1Neck 48.28,513 1:41.37,518 1/4 1:56.29,5,1:56.29,4,1/1/2019,149759,4.0
25,Bronx Sandman,3Neck,42 24.30,41/4 48.80,6 1:43.98,6 2:00.48,6,2:00.48,5,1/1/2019,149759,4.0
26,Carrera Cat,31/4,12 1/4 23.70,14 47.91,17 3/4 1:13.05,19 1/2 1:26.06,1,1:26.06,6,1/1/2019,149760,5.0
27,Gentle Annie,1Head,43/4 24.26,55 1/4 48.88,36 1:14.42,21 3/4 1:27.84,2,1:27.84,8,1/1/2019,149760,5.0
28,Passporttovictory,2Head,52 3/4 24.39,31/2 48.63,2Neck 1:14.40,33 1/2 1:28.20,3,1:28.20,1,1/1/2019,149760,5.0
29,Miss Marion,62 1/2,65 1/2 24.85,62 1/4 49.69,53/4 1:15.64,46 1/4 1:28.85,4,1:28.85,5,1/1/2019,149760,5.0


In [ ]:
len(urls_list)

In [ ]:
len(urls_list)